In [ ]:
!pip install datasets
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 8.2 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 68.0 MB/s 
     |████████████████████████████████| 86 kB 8.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 140 kB 70.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 127 kB 70.3 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 
     |████████████████████████████████| 144 kB 74.8 MB/s 
     |████████████████████████████████| 94 kB 4.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
     

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
train_dataset  = load_dataset('squad_v2', split="train")
valid_dataset = load_dataset('squad_v2', split="validation")

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


In [ ]:
mini_train_data = train_dataset.select(range(1000))

In [ ]:
mini_train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1000
})

In [ ]:
ans_text = [i["text"][0] for i in mini_train_data["answers"]]

In [ ]:
mini_train_data = mini_train_data.add_column("ans_text", ans_text)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-fc1fbce32e118d45.arrow


In [ ]:
# Tip: By now, install transformers from source

from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
mini_train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'ans_text'],
    num_rows: 1000
})

In [ ]:
input_texts = ["answer: "+ mini_train_data["ans_text"][ind] +"  context: "+ mini_train_data["context"][ind]+" </s>" for ind,_ in enumerate(mini_train_data)]

In [ ]:
input_texts

In [ ]:
mini_train_data = mini_train_data.add_column("input_texts", input_texts)

In [ ]:
def tokenize_function(examples):
    model_input = tokenizer(examples["input_texts"], truncation=True, padding="max_length")
    label = tokenizer(examples["question"], truncation=True, padding="max_length")
    model_input["labels"] = label["input_ids"]
    return model_input

In [ ]:
train_ds = mini_train_data.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-dc78baa25760c350.arrow


In [ ]:
mini_train_data['question']

In [ ]:
train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [ ]:
train_ds["attention_mask"]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
train_ds = train_ds.remove_columns(['id', 'title', 'context', 'question', 'answers', 'ans_text', 'input_texts'])

In [ ]:
train_ds.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=1)
eval_dataloader = DataLoader(train_ds, batch_size=1)

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["input_texts"], padding="max_length", truncation=True)
# train_ds = mini_train_data.map(tokenize_function, batched=True)
# train_ds

In [ ]:
# tokenized_dataset = train_ds.remove_columns(["title", "id",])

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=1)
eval_dataloader = DataLoader(train_ds, batch_size=1)

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
device

device(type='cuda')

In [ ]:
input  = {k: v.to(device) for k, v in train_dataloader.__iter__().__next__().items()}
outputs = model(**input)

In [ ]:
outputs

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # print(loss)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [ ]:
def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt').to(device)

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)
  # return output[0]
  return tokenizer.decode(output[0])

context = "Abdominal pain, also known as a stomach ache, is a symptom associated with both non-serious and serious medical issues."
answer = "both non-serious and serious medical issues"

get_question(answer, context)

'<pad><pad><pad><pad><pad><pad> question: What are symptoms of abdominal pain associated with?</s>'

In [ ]:
# train_ds["input_texts"]

In [ ]:
tokenizer.decode(train_ds["input_ids"][0])

'answer: in the late 1990s context: Beyoncé Giselle Knowles-Carter (/bi<unk>j<unk>nse<unk>/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". </s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p